In [1]:
import pandas as pd
import numpy as np
import torch
import os

import nlpsig
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search
)

seed = 2023

In [2]:
output_dir = "client_talk_type"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [3]:
anno_mi = pd.read_csv("AnnoMI-full.csv")
anno_mi["datetime"] = pd.to_datetime(anno_mi["timestamp"])
anno_mi = anno_mi.drop(columns=["video_title", "video_url"])
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-23 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-23 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-23 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-23 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-23 00:00:34


In [4]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [5]:
anno_mi["main_therapist_behaviour"].value_counts() / anno_mi["interlocutor"].value_counts()["therapist"]

other              0.313947
question           0.286258
reflection         0.251538
therapist_input    0.148257
Name: main_therapist_behaviour, dtype: float64

In [6]:
anno_mi["client_talk_type"].value_counts() / anno_mi["interlocutor"].value_counts()["client"]

neutral    0.627063
change     0.248030
sustain    0.124907
Name: client_talk_type, dtype: float64

In [7]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [8]:
anno_mi["topic"].value_counts()

reducing alcohol consumption                                                          2326
more exercise / increasing activity                                                   2034
reducing recidivism                                                                   1303
reducing drug use                                                                     1104
diabetes management                                                                    948
smoking cessation                                                                      923
smoking cessation                                                                      541
taking medicine / following medical procedure                                          448
asthma management                                                                      431
avoiding DOI                                                                           394
changing approach to disease                                                           315

In [9]:
len(anno_mi["transcript_id"].unique())

133

## Only considering client for now...

In [10]:
client_index = [isinstance(x, str) for x in anno_mi["client_talk_type"]]
sum(client_index)

6725

In [11]:
y_data = anno_mi["client_talk_type"][client_index]
y_data.shape

(6725,)

In [12]:
label_to_id = {y_data.unique()[i]: i for i in range(len(y_data.unique()))}
id_to_label = {v: k for k, v in label_to_id.items()}

In [13]:
y_data = [label_to_id[x] for x in y_data]

In [14]:
output_dim = len(label_to_id.values())
output_dim

3

## Obtaining SBERT Embeddings

We can use the `SentenceEncoder` class within `nlpsig` to obtain sentence embeddings from a model. This class uses the [`sentence-transformer`](https://www.sbert.net/docs/package_reference/SentenceTransformer.html) package and here, we have use the pre-trained `all-MiniLM-L12-v2` model by passing this name as a string to the class - alternative models can be found [here](https://www.sbert.net/docs/pretrained_models.html).

We can pass our dataframe and the column name which stores the sentences that we wish to encode along with the model name into the constructor of the class to initialise our sentence encoder as follows:

In [15]:
sbert_embeddings = np.load(f"{output_dir}/anno_mi_client_sentence_embeddings_384.npy")

## LSTM baseline

In [16]:
num_epochs = 100
hidden_dim_sizes = [200, 300]
num_layers = 1
bidirectional = True
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [17]:
size = 20
lstm_history, best_lstm_history, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    window_sizes=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type/lstm_history_20_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type/lstm_history_20_focal_2_best_model.csv


In [26]:
lstm_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.595390,0.469591,"[0.7165205813824476, 0.35767195767195764, 0.33...",None,0.595390,0.469591,"[0.7165205813824476, 0.35767195767195764, 0.33...",20,1,True,200,0.5,0.0010,0,2,True,0
0,focal,0.602677,0.483219,"[0.7221850613154961, 0.3624113475177305, 0.365...",None,0.602677,0.483219,"[0.7221850613154961, 0.3624113475177305, 0.365...",20,1,True,200,0.5,0.0010,1,2,True,0
0,focal,0.598662,0.464955,"[0.7225395430579965, 0.3476990504017531, 0.324...",None,0.598662,0.464955,"[0.7225395430579965, 0.3476990504017531, 0.324...",20,1,True,200,0.5,0.0010,12,2,True,0
0,focal,0.604907,0.493479,"[0.7198280737473136, 0.3936486933509758, 0.366...",None,0.604907,0.493479,"[0.7198280737473136, 0.3936486933509758, 0.366...",20,1,True,200,0.5,0.0010,123,2,True,0
0,focal,0.607732,0.474183,"[0.7274320771253286, 0.3717630365377793, 0.323...",None,0.607732,0.474183,"[0.7274320771253286, 0.3717630365377793, 0.323...",20,1,True,200,0.5,0.0010,1234,2,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.591078,0.466776,"[0.7131175086815279, 0.36548913043478265, 0.32...",None,0.591078,0.466776,"[0.7131175086815279, 0.36548913043478265, 0.32...",20,1,True,300,0.1,0.0005,0,2,True,17
0,focal,0.586468,0.465507,"[0.7077026572485704, 0.3559033684926846, 0.332...",None,0.586468,0.465507,"[0.7077026572485704, 0.3559033684926846, 0.332...",20,1,True,300,0.1,0.0005,1,2,True,17
0,focal,0.591078,0.473001,"[0.7128802442609974, 0.36794048021643555, 0.33...",None,0.591078,0.473001,"[0.7128802442609974, 0.36794048021643555, 0.33...",20,1,True,300,0.1,0.0005,12,2,True,17
0,focal,0.600149,0.473570,"[0.7203455532174105, 0.36509040333796944, 0.33...",None,0.600149,0.473570,"[0.7203455532174105, 0.36509040333796944, 0.33...",20,1,True,300,0.1,0.0005,123,2,True,17


In [21]:
best_lstm_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.584833,0.493681,"[0.6924899312959015, 0.42083333333333334, 0.36...",None,0.584833,0.493681,"[0.6924899312959015, 0.42083333333333334, 0.36...",1,True,200,0.5,0.0005,0,2,True
0,focal,0.590929,0.481947,"[0.7059092467710596, 0.39210611452604344, 0.34...",None,0.590929,0.481947,"[0.7059092467710596, 0.39210611452604344, 0.34...",1,True,200,0.5,0.0005,1,2,True
0,focal,0.593309,0.486329,"[0.7081604426002767, 0.40250391236306726, 0.34...",None,0.593309,0.486329,"[0.7081604426002767, 0.40250391236306726, 0.34...",1,True,200,0.5,0.0005,12,2,True
0,focal,0.597323,0.477945,"[0.7161434977578475, 0.3735224586288416, 0.344...",None,0.597323,0.477945,"[0.7161434977578475, 0.3735224586288416, 0.344...",1,True,200,0.5,0.0005,123,2,True
0,focal,0.607138,0.487799,"[0.7231095722104321, 0.3880597014925373, 0.352...",None,0.607138,0.487799,"[0.7231095722104321, 0.3880597014925373, 0.352...",1,True,200,0.5,0.0005,1234,2,True


In [18]:
best_lstm_history["f1"].mean()

0.48553996318887094

In [19]:
np.stack(best_lstm_history["f1_scores"]).mean(axis=0)

array([0.70916254, 0.3954051 , 0.35205225])

In [27]:
size = 10
lstm_history, best_lstm_history, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    window_sizes=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type/lstm_history_10_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type/lstm_history_10_focal_2_best_model.csv


In [28]:
lstm_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.565353,0.438263,"[0.6937485894831866, 0.33299899699097285, 0.28...",None,0.565353,0.438263,"[0.6937485894831866, 0.33299899699097285, 0.28...",10,1,True,200,0.5,0.0010,0,2,True,0
0,focal,0.579033,0.447113,"[0.7034544638851504, 0.3638752052545156, 0.274...",None,0.579033,0.447113,"[0.7034544638851504, 0.3638752052545156, 0.274...",10,1,True,200,0.5,0.0010,1,2,True,0
0,focal,0.605799,0.413250,"[0.7378940861322892, 0.2838309422256689, 0.218...",None,0.605799,0.413250,"[0.7378940861322892, 0.2838309422256689, 0.218...",10,1,True,200,0.5,0.0010,12,2,True,0
0,focal,0.574424,0.438938,"[0.7007874015748031, 0.35734331150608045, 0.25...",None,0.574424,0.438938,"[0.7007874015748031, 0.35734331150608045, 0.25...",10,1,True,200,0.5,0.0010,123,2,True,0
0,focal,0.584535,0.427958,"[0.7167316861725184, 0.32014260249554366, 0.24...",None,0.584535,0.427958,"[0.7167316861725184, 0.32014260249554366, 0.24...",10,1,True,200,0.5,0.0010,1234,2,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.580223,0.427867,"[0.7106986899563319, 0.32700135685210313, 0.24...",None,0.580223,0.427867,"[0.7106986899563319, 0.32700135685210313, 0.24...",10,1,True,300,0.1,0.0005,0,2,True,17
0,focal,0.590781,0.432105,"[0.718297688485634, 0.33935264054514475, 0.238...",None,0.590781,0.432105,"[0.718297688485634, 0.33935264054514475, 0.238...",10,1,True,300,0.1,0.0005,1,2,True,17
0,focal,0.583643,0.445401,"[0.7104418115380357, 0.3488372093023256, 0.276...",None,0.583643,0.445401,"[0.7104418115380357, 0.3488372093023256, 0.276...",10,1,True,300,0.1,0.0005,12,2,True,17
0,focal,0.581710,0.436894,"[0.7077771649200222, 0.35120994113799875, 0.25...",None,0.581710,0.436894,"[0.7077771649200222, 0.35120994113799875, 0.25...",10,1,True,300,0.1,0.0005,123,2,True,17


In [29]:
best_lstm_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.582454,0.438405,"[0.7108143607705779, 0.32909604519774016, 0.27...",None,0.582454,0.438405,"[0.7108143607705779, 0.32909604519774016, 0.27...",1,True,300,0.5,0.0005,0,2,True
0,focal,0.577993,0.452522,"[0.6995912806539509, 0.3770287141073658, 0.280...",None,0.577993,0.452522,"[0.6995912806539509, 0.3770287141073658, 0.280...",1,True,300,0.5,0.0005,1,2,True
0,focal,0.585428,0.446715,"[0.7168648172611184, 0.31303698169592825, 0.31...",None,0.585428,0.446715,"[0.7168648172611184, 0.31303698169592825, 0.31...",1,True,300,0.5,0.0005,12,2,True
0,focal,0.608625,0.431839,"[0.7368969469625924, 0.3072196620583717, 0.251...",None,0.608625,0.431839,"[0.7368969469625924, 0.3072196620583717, 0.251...",1,True,300,0.5,0.0005,123,2,True
0,focal,0.567584,0.442752,"[0.6938498131158681, 0.3469049983449189, 0.2875]",None,0.567584,0.442752,"[0.6938498131158681, 0.3469049983449189, 0.2875]",1,True,300,0.5,0.0005,1234,2,True


In [30]:
best_lstm_history["f1"].mean()

0.44244645933164994

In [31]:
np.stack(best_lstm_history["f1_scores"]).mean(axis=0)

array([0.71160344, 0.33465728, 0.28107865])

In [32]:
size = 50
lstm_history, best_lstm_history, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    window_sizes=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 